In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Create Policy to Restrict Transactions in a Period

This notebook shows how to create a policy to restrict transactions in between 2 effective dates by denying any activity to all portfolios in a given scope.

Attributes
----------
policies
transactions
"""

toggle_code("Hide docstring")

/opt/conda/lib/python3.9/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# Create Policy to Restrict Transactions in a Period

This notebook shows how to create a policy to restrict transactions in between 2 effective dates by denying any activity to all portfolios in a given scope.

![Timeline](timeline.JPG)

## 1. Imports and Setup

In [2]:
# Import LUSID
import lusid
import finbourne_access

#import lusid_sample_data as import_data
from lusid.utilities import ApiClientFactory as LusidApiClientFactory
from finbourne_access.utilities import ApiClientFactory as AccessApiClientFactory
from lusidjam import RefreshingToken

# Import Libraries
import os
import json
from IPython.display import Image

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

access_api_factory = AccessApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

# print("LUSID Environment Initialised")
# print(
#     "LUSID SDK Version: ",
#     access_api_factory.build(finbourne_access.api.ApplicationMetadataApi)
#     .get_lusid_versions()
#     .build_version,
# )

# Define APIs
access_api = access_api_factory.build(finbourne_access.api.PoliciesApi)


## 2. Define and Create Policy

This policy denies any update, delete or add activity to portfolios in the provided given scope during the provided time frame. This prevents any transactions from being added, updated or deleted in this time frame.

This policy upon being created can be added to a role and applied to a user to add these restrictions to them.

In [3]:
# Define variables for use in the policy request body
scope = "ibor"
policy_code = "block-transactions"
policy_description = "Policy to block transactions in a given date range"
effective_from = "2022-01-01T00:00:00.0000000+00:00"
effective_to = "2022-04-01T00:00:00.0000000+00:00"


# Define policy request body
request_body = {
    "code": policy_code,
    "description": policy_description,
    "applications": [
        "LUSID"
    ],
    "grant": "Deny",
    "selectors": [
        {
            "idSelectorDefinition": {
                "identifier": {
                    "code": "*",
                    "scope": "ibor"
                },
                "actions": [
                    {
                        "scope": "default",
                        "activity": "Update",
                        "entity": "Portfolio"
                    },
                    {
                        "scope": "default",
                        "activity": "Delete",
                        "entity": "Portfolio"
                    },
                    {
                        "scope": "default",
                        "activity": "Add",
                        "entity": "Portfolio"
                    }
                ]
            }
        }
    ],
    "for": [
        {
            "effectiveRange": {
                "from": effective_from,
                "to": effective_to
            }
        }
    ],
    "when": {
        "activate": "0001-01-01T00:00:00.0000000+00:00",
        "deactivate": "9999-12-31T23:59:59.9999990+00:00"
    }
}

In [4]:
try:
    response = access_api.create_policy(request_body)
    response
except finbourne_access.ApiException as e:
    print(json.loads(e.body)["title"])

Policy already exists
